In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime  
import time
from hashlib import sha1
import hmac
from wsgiref.handlers import format_date_time
from datetime import datetime
from time import mktime
import base64

In [2]:
app_id = '2d16769d481b4ab69ec6ecb0ccd0672b'
app_key = 'dbr3zEbzwX8cj1Yv7ZxGzb4IpAc'
class Auth():

    def __init__(self, app_id, app_key):
        self.app_id = app_id
        self.app_key = app_key

    def get_auth_header(self):
        xdate = format_date_time(mktime(datetime.now().timetuple()))
        hashed = hmac.new(self.app_key.encode('utf8'), ('x-date: ' + xdate).encode('utf8'), sha1)
        signature = base64.b64encode(hashed.digest()).decode()

        authorization = 'hmac username="' + self.app_id + '", ' + \
                        'algorithm="hmac-sha1", ' + \
                        'headers="x-date", ' + \
                        'signature="' + signature + '"'
        return {
            'Authorization': authorization,
            'x-date': format_date_time(mktime(datetime.now().timetuple())),
            'Accept - Encoding': 'gzip'
        }

a = Auth(app_id, app_key)
r=requests.get("https://ptx.transportdata.tw/MOTC/v2/Bus/Station/InterCity?$top=1&$format=xml", headers= a.get_auth_header())
r.close()
soup = BeautifulSoup(r.text, "html.parser")
print(soup.prettify())

In [4]:
a = Auth(app_id, app_key)
r=requests.get("https://ptx.transportdata.tw/MOTC/v2/Bus/Station/InterCity?$top=1&$format=xml", headers= a.get_auth_header())
r.close()
soup = BeautifulSoup(r.text, "html.parser")
for i in soup.find_all('busstation'): #每次跑一個站的資料
    #print(soup.prettify())
    #print(j)
    #for j in i.find_all("stops"):
    stationuid = [stationuid.text for stationuid in i.find_all('stationuid')]

    stationid = [stationid.text for stationid in i.find_all('stationid')]

    StopName_zh_tw = [i.zh_tw.text for i in i.find_all('stationname')]

    StopName_En = [en.text for en in soup.find_all('en')]

    PositionLat = [positionlat.text for positionlat in i.find_all('positionlat')]

    PositionLon = [positionlon.text for positionlon in i.find_all('positionlon')]

    geohash = [geohash.text for geohash in i.find_all('geohash')]
    
    stationgroupid = [stationgroupid.text for stationgroupid in i.find_all('stationgroupid')]
    
    #Stopaddress = [stopaddress.text for stopaddress in i.find_all('stationaddress')]

    UpdateTime = [updatetime.text for updatetime in i.find_all('updatetime')]
    VersionID = [versionid.text for versionid in i.find_all('versionid')]

    BusStation={'站牌唯一識別代碼':stationuid,'站牌代碼(為原資料內碼)':stationid,'站牌中文名稱':StopName_zh_tw,'站牌英文名稱':StopName_En
        ,"站牌緯度(WGS84)":PositionLat,"站牌經度(WGS84)":PositionLon
        ,"地理空間編碼":geohash,"站牌所屬的組站位ID":stationgroupid,"資料更新日期時間":UpdateTime,"資料版本編號":VersionID}
    #print(Bus_information)
    BusStation = pd.DataFrame.from_dict(BusStation, orient='index')
    BusStation=BusStation.transpose()#把欄位顛倒過來

    #合併經過公車
    stationuid = [stationuid.text for stationuid in i.find_all('stationuid')]
    stopuid = [i.stopuid.text for i in i.find_all('stop')]
    stopid = [i.stopid.text for i in i.find_all('stop')]
    stopname_zh_tw=[i.stopname.zh_tw.text for i in i.find_all('stop')]
    stopname_en = [i.stopname.en.text for i in i.find_all('stop')]
    routeuid = [i.routeuid.text for i in i.find_all('stop')]
    routeid=[i.routeid.text for i in i.find_all('stop')]
    routename_zh_tw = [i.routename.zh_tw.text for i in i.find_all('stop')]
    routename_en=[i.routename.en.text for i in i.find_all('stop')]
    Bus_connection={'站牌唯一識別代碼':stationuid,'經此站牌唯一識別代碼':stopuid,'經此站牌代碼(為原資料內碼)':stopid,'經此站牌中文名稱':stopname_zh_tw
            ,"經此站牌英文名稱":stopname_en ,"經此路線識別代碼":routeuid,"經此路線代碼":routeid
            ,"經此路線中文":routename_zh_tw,"經此路線英文":routename_en}
    #print(_all)
    df = pd.DataFrame.from_dict(Bus_connection, orient='index')
    df=df.transpose()#把欄位顛倒過來
    df=df.fillna(method='pad')#用前一個值彌補後一個值
    _all=pd.merge(BusStation, df,on="站牌唯一識別代碼")#完整資料
    #print(_all)
    #Bus_information=pd.concat([Bus_information, _all])
    #break

    #break

In [5]:
_all

,站牌唯一識別代碼,站牌代碼(為原資料內碼),站牌中文名稱,站牌英文名稱,站牌緯度(WGS84),站牌經度(WGS84),地理空間編碼,站牌所屬的組站位ID,資料更新日期時間,資料版本編號,經此站牌唯一識別代碼,經此站牌代碼(為原資料內碼),經此站牌中文名稱,經此站牌英文名稱,經此路線識別代碼,經此路線代碼,經此路線中文,經此路線英文
0,THB102582,102582,東行站位,Taitung Hospital,22.7575,121.14978,wsn1kyfs4,950－039,2021-01-31T07:34:50+08:00,785,THB135427,135427,署東醫院,Taitung Hospital,THB8101,8101,8101,8101
1,THB102582,102582,東行站位,Taitung Hospital,22.7575,121.14978,wsn1kyfs4,950－039,2021-01-31T07:34:50+08:00,785,THB135427,135427,署東醫院,Taitung Hospital,THB8103,8103,8103,8103
2,THB102582,102582,東行站位,Taitung Hospital,22.7575,121.14978,wsn1kyfs4,950－039,2021-01-31T07:34:50+08:00,785,THB135427,135427,署東醫院,Taitung Hospital,THB8109,8109,8109,8109
3,THB102582,102582,東行站位,Taitung Hospital,22.7575,121.14978,wsn1kyfs4,950－039,2021-01-31T07:34:50+08:00,785,THB135427,135427,署東醫院,Taitung Hospital,THB8113,8113,8113,8113
4,THB102582,102582,東行站位,Taitung Hospital,22.7575,121.14978,wsn1kyfs4,950－039,2021-01-31T07:34:50+08:00,785,THB135427,135427,署東醫院,Taitung Hospital,THB8117,8117,8117,8117
5,THB102582,102582,東行站位,Taitung Hospital,22.7575,121.14978,wsn1kyfs4,950－039,2021-01-31T07:34:50+08:00,785,THB135427,135427,署東醫院,Taitung Hospital,THB8118,8118,8118,8118
6,THB102582,102582,東行站位,Taitung Hospital,22.7575,121.14978,wsn1kyfs4,950－039,2021-01-31T07:34:50+08:00,785,THB135427,135427,署東醫院,Taitung Hospital,THB8122,8122,8122,8122
7,THB102582,102582,東行站位,Taitung Hospital,22.7575,121.14978,wsn1kyfs4,950－039,2021-01-31T07:34:50+08:00,785,THB135427,135427,署東醫院,Taitung Hospital,THB8128,8128,8128,8128
8,THB102582,102582,東行站位,Taitung Hospital,22.7575,121.14978,wsn1kyfs4,950－039,2021-01-31T07:34:50+08:00,785,THB135427,135427,署東醫院,Taitung Hospital,THB8132,8132,8132,8132
9,THB102582,102582,東行站位,Taitung Hospital,22.7575,121.14978,wsn1kyfs4,950－039,2021-01-31T07:34:50+08:00,785,THB135427,135427,署東醫院,Taitung Hospital,THB8135,8135,8135,8135


In [10]:
_all.to_csv(r'C:\Users\Road_passenger_transport.csv', index=False,encoding="utf_8_sig" )

In [ ]:
BusStation {
StationUID (String): 站位唯一識別代碼，規則為 {業管機關簡碼} + {StationID}，其中 {業管機關簡碼} 可於Authority API中的AuthorityCode欄位查詢 ,
StationID (String): 站位代碼 ,
StationName (NameType): 站位名稱 ,
StationPosition (PointType): 站位位置 ,
StationAddress (String, optional): 站位地址 ,
StationGroupID (String): 站牌所屬的組站位ID ,
Stops (Array[StationStop], optional): 站牌與所行經此站牌之路線列表(資料會對路線展開，因此可能會有重複的站牌資料) ,
UpdateTime (DateTime): 資料更新日期時間(ISO8601格式:yyyy-MM-ddTHH:mm:sszzz) ,
VersionID (Int32): 資料版本編號
}
NameType {
Zh_tw (String, optional): 中文繁體名稱 ,
En (String, optional): 英文名稱
}
PointType {
PositionLat (number, optional): 位置緯度(WGS84) ,
PositionLon (number, optional): 位置經度(WGS84) ,
GeoHash (String, optional): 地理空間編碼
}
StationStop {
StopUID (String): 站牌唯一識別代碼，規則為 {業管機關簡碼} + {StopID}，其中 {業管機關簡碼} 可於Authority API中的AuthorityCode欄位查詢 ,
StopID (String): 地區既用中之站牌代碼(為原資料內碼) ,
StopName (NameType): 站牌名稱 ,
RouteUID (String): 路線唯一識別代碼，規則為 {業管機關簡碼} + {RouteID}，其中 {業管機關簡碼} 可於Authority API中的AuthorityCode欄位查詢 ,
RouteID (String): 地區既用中之路線代碼(為原資料內碼) ,
RouteName (NameType): 路線名稱
}